In [40]:
pip install --upgrade keras

Note: you may need to restart the kernel to use updated packages.


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import spacy
from spacy import displacy
import nltk
from nltk.corpus import stopwords
import re
import en_vectors_web_lg
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

In [3]:
df_train = pd.read_csv(r"C:\Users\UserC\Desktop\CORONA/Corona_NLP_train.csv", encoding='latin1')
df_test = pd.read_csv(r"C:\Users\UserC\Desktop\CORONA/Corona_NLP_test.csv", encoding='latin1')
df = pd.concat([df_train, df_test], axis=0)
df['TweetAt'] = pd.to_datetime(df['TweetAt'])
print(df.head())
print(df.info())

   UserName  ScreenName   Location    TweetAt  \
0      3799       48751     London 2020-03-16   
1      3800       48752         UK 2020-03-16   
2      3801       48753  Vagabonds 2020-03-16   
3      3802       48754        NaN 2020-03-16   
4      3803       48755        NaN 2020-03-16   

                                       OriginalTweet           Sentiment  
0  @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...             Neutral  
1  advice Talk to your neighbours family to excha...            Positive  
2  Coronavirus Australia: Woolworths to give elde...            Positive  
3  My food stock is not the only one which is emp...            Positive  
4  Me, ready to go at supermarket during the #COV...  Extremely Negative  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 44955 entries, 0 to 3797
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   UserName       44955 non-null  int64     

In [4]:
df['Location'].fillna(value = 0,  inplace = True) 
non_location = df[df['Location']==0]

In [5]:
nlp = spacy.load('en_vectors_web_lg')

In [6]:
tweets = df.OriginalTweet
tweets.head().tolist()

['@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8',
 'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
 'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P',
 "My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",
 "Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thin

In [7]:
tweets = tweets.str.lower()
# remove 't.co/' links
tweets = tweets.apply(lambda x : re.sub(r'https://t.co\/[^\s]+', '', x, flags=re.MULTILINE))
# removing special characters and numbers
tweets = tweets.apply(lambda x : re.sub("[^a-z\s]","",x) )
# removing stopwords
stopwords = set(stopwords.words("english"))
tweets = tweets.apply(lambda x : " ".join(word for word in x.split() if word not in stopwords ))


In [8]:
df['tweets'] = tweets
df_simplified = df.copy()
df_main = df.copy()
df_pipe = df_main.copy()

df["Sentiment"].replace({'Extremely Negative': -2, 'Negative': -1, 'Neutral': 0, 'Positive': 1, 'Extremely Positive': 2}, inplace=True)

df_two_val = df.copy()
df_two_val = df_two_val[df_two_val.Sentiment != 0]
df_two_val["Sentiment"].replace({-2: 0,-1: 0,2: 1,1: 1}, inplace=True)

df_simplified["Sentiment"].replace({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Extremely Positive': 2}, inplace=True)

In [11]:
doc_df = nlp.pipe(tweets)
tweets_vector_df = np.array([tweet.vector for tweet in doc_df])


tweets_df2val = df_two_val.tweets
doc_df2val = nlp.pipe(tweets_df2val)
tweets_vector_df2val = np.array([tweet.vector for tweet in doc_df2val])

tweets_df3val = df_simplified.tweets
doc_df3val = nlp.pipe(tweets_df3val)
tweets_vector_df3val = np.array([tweet.vector for tweet in doc_df3val])

In [15]:
X = tweets_vector_df
y = df["Sentiment"]
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))
#_____________________________________#
#_____________________________________#

X2 = tweets_vector_df2val
y2 = df_two_val["Sentiment"]
X2_train,X2_test, y2_train, y2_test = train_test_split(X2,y2, stratify=y2, test_size=0.2, random_state=0)

#model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X2_train, y2_train)
y2_pred = model.predict(X2_test)
print("Accuracy on df_two_val test data is : %0.2f" %(accuracy_score(y2_test, y2_pred)*100))
y2_train_pred = model.predict(X2_train)
print("Accuracy on df_two_val train data is : %0.2f" %(accuracy_score(y2_train, y2_train_pred)*100))
#_____________________________________#
#_____________________________________#

X3 = tweets_vector_df3val
y3 = df["Sentiment"]
X3_train,X3_test, y3_train, y3_test = train_test_split(X3,y3, stratify=y3, test_size=0.2, random_state=0)

#model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("Accuracy on df_simplified test data is : %0.2f" %(accuracy_score(y3_test, y3_pred)*100))
y3_train_pred = model.predict(X3_train)
print("Accuracy on df_simplified train data is : %0.2f" %(accuracy_score(y3_train, y3_train_pred)*100))

Accuracy on df test data is : 48.35
Accuracy on df train data is : 50.27
Accuracy on df_two_val test data is : 78.51
Accuracy on df_two_val train data is : 79.09
Accuracy on df_simplified test data is : 48.35
Accuracy on df_simplified train data is : 50.27


In [ ]:
#What I learnt from above is that Logistic regression, as I thought works quite good for 2 values

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [16]:

y_neighbours = df["Sentiment"].values
knn = KNeighborsClassifier(n_neighbors=6)

# Fit the classifier to the data
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.3, random_state=0)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy on test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = knn.predict(X_train)
print("Accuracy on train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))



Accuracy on test data is : 36.58
Accuracy on train data is : 56.16


In [17]:
knn4 = KNeighborsClassifier(n_neighbors=4)

knn4.fit(X_train, y_train)
y_train_pred = knn.predict(X_train)
print("Accuracy on train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))


Accuracy on train data is : 56.16


In [18]:
knn3 = KNeighborsClassifier(n_neighbors=3)

knn3.fit(X_train, y_train)
y_train_pred = knn3.predict(X_train)
print("Accuracy on train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on train data is : 62.69


In [14]:
X = tweets_vector_df
y = pd.get_dummies(df_main["Sentiment"])['Extremely Negative'] 
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 88.05
Accuracy on df train data is : 88.52


In [20]:
X = tweets_vector_df
y = pd.get_dummies(df_main["Sentiment"])['Negative'] 
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 74.63
Accuracy on df train data is : 74.97


In [24]:
X = tweets_vector_df
y = pd.get_dummies(df_main["Sentiment"])['Neutral'] 
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 84.77
Accuracy on df train data is : 85.14


In [25]:
X = tweets_vector_df
y = pd.get_dummies(df_main["Sentiment"])['Positive'] 
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 71.56
Accuracy on df train data is : 71.93


In [26]:
X = tweets_vector_df
y = pd.get_dummies(df_main["Sentiment"])['Extremely Positive'] 
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

model = LogisticRegression(C=0.2, max_iter=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = model.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 86.94
Accuracy on df train data is : 87.01


In [30]:
estimators = [('reduce_dim', PCA()), ('clf', SGDClassifier())]
pipe = Pipeline(estimators)

X = tweets_vector_df
y = df["Sentiment"]
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = pipe.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

Accuracy on df test data is : 41.32
Accuracy on df train data is : 42.85


In [ ]:
pipe = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
X = tweets_vector_df
y = df["Sentiment"]
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print("Accuracy on df test data is : %0.2f" %(accuracy_score(y_test, y_pred)*100))
y_train_pred = pipe.predict(X_train)
print("Accuracy on df train data is : %0.2f" %(accuracy_score(y_train, y_train_pred)*100))

In [45]:
df_auxiliary = pd.get_dummies(df_main["Sentiment"]) 
binary_matrix = df_auxiliary.values

X = tweets_vector_df
y = binary_matrix
X_train,X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=0)

ohe = OneHotEncoder()
ohe.fit(X_train)
X_train_tf = ohe.transform(X_train)
X_test_tf = ohe.transform(X_test)

le = LabelEncoder()
le.fit(y_train)
y_train_tf = le.transform(y_train)
y_test_tf = le.transform(y_test)

model = Sequential()
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=100, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))



ValueError: Found unknown categories [-0.76677, -0.65017, -0.64432, -0.58409667, -0.5793058, -0.57105, -0.5603025, -0.5574567, -0.5520247, -0.52742, -0.51239496, -0.50086296, -0.49885666, -0.49600998, -0.49563116, -0.49074683, -0.49011704, -0.48959342, -0.48953804, -0.48535156, -0.48453, -0.48389688, -0.4826651, -0.4813731, -0.474142, -0.46928397, -0.4669109, -0.46660116, -0.4665516, -0.4635087, -0.4629182, -0.46172738, -0.4612756, -0.4611827, -0.460848, -0.46022573, -0.45779416, -0.45651138, -0.45609266, -0.45605472, -0.4553683, -0.45483425, -0.45347303, -0.45324495, -0.45237797, -0.45060998, -0.44730985, -0.4454481, -0.44410896, -0.44311672, -0.44107124, -0.43751276, -0.43723652, -0.43721846, -0.43574452, -0.43552202, -0.4354975, -0.43492097, -0.43484756, -0.434756, -0.4337983, -0.43339667, -0.43305567, -0.43300915, -0.43256137, -0.42886782, -0.42868495, -0.4283845, -0.42831418, -0.4282, -0.42815, -0.42683953, -0.42612302, -0.425308, -0.424536, -0.42354524, -0.42252752, -0.42227763, 

In [ ]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))